### Activity 2- API call to fetch the character's data dynamically

In [11]:
import requests
import hashlib
import json
import pandas as pd
import warnings
from warnings import filterwarnings
warnings.filterwarnings("ignore")

In [4]:
public_key='796418b3ab81b409d0968e5eddc9145a'
private_key='f1af59a597179df791d6a9030fad51c441ce9a1b'
ts=1
hash= hashlib.md5((str(ts)+private_key+public_key).encode()).hexdigest()
address = 'https://gateway.marvel.com:443/v1/public/characters'

In [5]:
parameters = {
    "apikey": public_key,
    "ts": ts,
    "hash": hash
}

#### First API call

In [ ]:
response =requests.get(address, params=parameters)
results = response.json()
print(json.dumps(results))
#first api call

#### Fetching all the charecters through a for loop

In [ ]:
from string import ascii_lowercase

charecters_df =pd.DataFrame()

for ch in list(ascii_lowercase):
    parameters={'ts':1,'apikey':public_key, 'hash':hash,'limit':100, 'nameStartsWith':ch}
    response =requests.get(address, params=parameters)
    results = response.json()
    df=pd.json_normalize(results['data'],['results'])
    df =df[['id','name','comics.available','events.available','stories.available','series.available']]
    charecters_df=charecters_df.append(df)

### Activity 3  -Defining function to generates the data frame


In [8]:
def df_characters(public_key,hash,nameStartsWith):
    for ch in list(ascii_lowercase):
        parameters={'ts':1,'apikey':public_key, 'hash':hash, 'nameStartsWith':nameStartsWith}
        response =requests.get(address, params=parameters)
    
        results = response.json()

        df=pd.json_normalize(results['data']['results'])
        df =df[['id','name','comics.available','events.available','stories.available','series.available']]
    return df

### Activity 4 - Creating a function to filter out the characters based on conditions!

In [9]:
#activity 4
def filter_char(df,column_name, filter_condition,filter_value):
    if column_name== 'name':
        filter_condition= 'starts_with'
        length =len(filter_value)
        return(df[df.name.str[:length]== filter_value])
    else:
        if filter_condition == 'equal_to':
            return(df[df[column_name]== int(filter_value)])
        elif filter_condition =='less_than':
            return(df[df[column_name]<int(filter_value)])
        elif filter_condition =='greater_than':
            return(df[df[column_name]>int(filter_value)])
        else:
            return('error')